In [2]:
def find_bigrams(input_list):
    return zip(input_list, input_list[1:])

# Source: https://stackoverflow.com/questions/32037893/numpy-fix-array-with-rows-of-different-lengths-by-filling-the-empty-elements-wi
# by user Divakar
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out    

def tokenize(s):
    return tokens_re.findall(s)

    #args s = tweets, lowercase=False(keeps as in tweets) / True(changes all to lowercase)
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

import tweepy
import csv
import string
import re
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle

from itertools import chain
from scipy import sparse
from IPython import display
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.svm import LinearSVC

#Auth details from Twitter app http://twitclassify.com/
CONSUMER_KEY = "90bt0hlKzf3zWi5aPJ5ELZOGL"
CONSUMER_SECRET = "H4ycskJVRjcesIaZBVUFdTClsucdMEyPcZyQT7Xjvou3LWwBg2"
ACCESS_KEY = "303590178-hw3mgzNW08xAvbrVW4Zm8VP4uKpU9qyfG01ynkzi"
ACCESS_SECRET = "6wJw3hGpFAgR77nk6Hw4UjCGi96OtOZhnXjm1iMcptQ3i"

authid = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
authid.set_access_token(ACCESS_KEY, ACCESS_SECRET)
myApi = tweepy.API(authid, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'if', 'i', '’', '“', '”', 'it', 'the', '…', 'us', ':)', '—',
                                                   ':/', 'il', '', '≠', '•']

# Source taken from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

data_train = np.genfromtxt("trainingmix.tsv", delimiter='\t', dtype= str)

twitHandle = data_train[:,1] # this gets the tweet handles [column B]
y = data_train[:,0]

### Lists ###
desc_train_set = []
li = []
li_desc = []
uniqueWords = []
uniqueDict= {}
convList = []
bigList = []
userCap = []
userList = []
user_id = []

userCap = userCap + [twitHandle[0]] # insert first element to list because 0 % 100 == 0 but should be treated as != 0
print(len(twitHandle))
# take every 100 users, put into nested list eg. [[first 100 users][second 100 users]...[n 100 users]]
for i in range(1,len(twitHandle)+1):
    if(i == len(twitHandle)):
        userList.append(userCap) # append 100 users into an index of userList
        
    elif (i % 100 != 0):
        userCap = userCap + [twitHandle[i]]
        
    elif (i % 100 == 0):
        userList.append(userCap) # append 100 users into an index of userList
        userCap = [] # empty userCap (user capacity) list
        userCap = userCap + [twitHandle[i]] # add current iteration, i, element into userCap

for i in range(len(userList)):

    comma_separated_string = ",".join(userList[i]) # convert to single string for passing
    user_objects = myApi.lookup_users(screen_names= [comma_separated_string]) # lookup_users limitation on 100 users per req
    user_id_cont = [user.id_str for user in user_objects] # takes each 100 user ids
    user_id.append(user_id_cont) #store for each 100 users
    user_id_cont = [] # empty list for next iteration

for i in range(len(user_id)):
# Gets the user_id from each user in twitHandle
    display.clear_output(wait=True) #refresh display
    print(i+1) # Show progress of user retrieval
    userProfile = myApi.lookup_users(user_ids=user_id[i])
    for user in userProfile:
       
        #user profile description
        terms_stop = [term for term in preprocess(user.description) 
                      if term not in stop and
                      not term.startswith(('#', '@', 'http', 'https'))]

        #stores all terms into a list
        li_desc = li_desc + terms_stop
        
        # separates each descriptions to their respective users
        # add list of desc for each user into desc train set ie. desc_train_set = [[desc 1][desc 2]...[desc n]]
        desc_train_set.append(terms_stop)

    # Append all unique unigram terms into uniqeWords list
    for j in range(len(li_desc)):
        for word in li_desc:
            if not word in uniqueWords:
                uniqueWords.append(word)

    # empty the list for next iteration    
    li_desc = []
    

bigrams = list(chain.from_iterable(desc_train_set))
bigrams = list(find_bigrams(bigrams))

# Append all unique bigram terms into uniqeWords list
for i in range(len(bigrams)):
        for word in bigrams:
            if not word in uniqueWords:
                uniqueWords.append(word)

# Initializes a key, value of (eg. {term: id} ) into dictionary
for i,j in enumerate(uniqueWords):
    uniqueDict[i+1] = (j,i)
    
for j in range(len(uniqueDict)): # to create a single dictionary
    uniqueDict[j+1] = uniqueDict[j+1][0]
    
print("Finished creating dictionary!")

# Save to .pickle
with open('profileDict.pkl', 'wb') as f:
    pickle.dump(uniqueDict, f)

inv_map = {v: k for k, v in uniqueDict.items()} # to reverse dict ordering
for index,word in enumerate(desc_train_set):         # iterate over set    
    for key, value in uniqueDict.items():
        if value in word:                      # see if word exists in dictionary
            convList.append(key)
    print(index)
    bigList.append(convList)
    convList = []
    
print("Done conversion!")
print(len(bigList))
desc_train_set = np.asarray(bigList)
print(desc_train_set)

print(desc_train_set.shape)
desc_train_set = numpy_fillna(desc_train_set) # Match dimensions of array with highest no. of feature
print(desc_train_set.shape)

# Save to .pickle
with open('profileColLen.pkl', 'wb') as f:
    pickle.dump(desc_train_set.shape[1], f)

#splits data into training and test set
trainX, testX, train_y, test_y = train_test_split(desc_train_set, y, test_size=0.5)
#SVM Classifier full data
   
# fit a SVM model to the data
modelSVM = LinearSVC(penalty='l2', loss='squared_hinge', dual=False, tol=0.0001, C=5.0, multi_class='ovr',
                     fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
                     random_state=None, max_iter=1000)
modelSVM.fit(trainX, train_y)
# Save to .pickle
with open('profileModel.pkl', 'wb') as f:
    pickle.dump(modelSVM, f)
    

5
Finished creating dictionary!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

In [3]:
from sklearn.svm import LinearSVC
import operator
import itertools


# Source: https://stackoverflow.com/questions/1518522/python-most-common-element-in-a-list
# by user Alex Martelli
def most_common(L):
    # get an iterable of (item, iterable) pairs
    SL = sorted((x, i) for i, x in enumerate(L))
    # print 'SL:', SL
    groups = itertools.groupby(SL, key=operator.itemgetter(0))
    # auxiliary function to get "quality" for an item
    def _auxfun(g):
        item, iterable = g
        count = 0
        min_index = len(L)
        for _, where in iterable:
            count += 1
            min_index = min(min_index, where)
        # print 'item %r, count %r, minind %r' % (item, count, min_index)
        return count, -min_index
    # pick the highest-count/earliest item
    result = max(groups, key=_auxfun)[0]
    return result

# Load from .pickle
with open('profileModel.pkl','rb') as f:
    modelSVM = pickle.load(f)

print(modelSVM)
# make predictions
expected = test_y
predicted = modelSVM.predict(testX)
# summarize the fit of the model
# Classification report

class_report = metrics.classification_report(expected, predicted)
print("Classification report \n", class_report)

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = \n", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)

LinearSVC(C=5.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
Community:  Academic
Classification report 
              precision    recall  f1-score   support

          1       0.71      0.83      0.77        54
          2       0.47      0.70      0.57        37
          3       0.37      0.39      0.38        49
          4       0.28      0.18      0.22        50
          5       0.43      0.35      0.39        60

avg / total       0.46      0.48      0.46       250

Confusion Metric = 
 [[45  8  1  0  0]
 [ 5 26  4  2  0]
 [ 5  7 19 10  8]
 [ 2 10  9  9 20]
 [ 6  4 18 11 21]]
Accuracy =  0.48
Sensitivity =  0.9
Specificity =  0.764705882353


In [5]:
from sklearn.naive_bayes import MultinomialNB

#Multinomial Naive Bayes Classifier full data

# fit a SVM model to the data
modelMNB = MultinomialNB()
modelMNB.fit(trainX, train_y)
print(modelMNB)
# make predictions
expected = test_y
predicted = modelMNB.predict(testX)
# summarize the fit of the model
# Classification report
print("Classification report ", metrics.classification_report(expected, predicted))

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = \n", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Classification report               precision    recall  f1-score   support

          1       0.50      0.02      0.04        54
          2       0.19      0.14      0.16        37
          3       0.22      0.45      0.30        49
          4       0.25      0.08      0.12        50
          5       0.30      0.53      0.38        60

avg / total       0.30      0.26      0.21       250

Confusion Metric = 
 [[ 1  6 20  6 21]
 [ 1  5 13  3 15]
 [ 0  7 22  1 19]
 [ 0  5 20  4 21]
 [ 0  3 23  2 32]]
Accuracy =  0.256
Sensitivity =  0.5
Specificity =  0.454545454545


In [6]:
from sklearn.linear_model import LogisticRegression

# fit a Logistic Regression model to the data
modelLR = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
                   intercept_scaling=1, class_weight=None, random_state=None,
                   solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

modelLR.fit(trainX, train_y)
# make predictions
expected = test_y
predicted = modelLR.predict(testX)
# summarize the fit of the model
# Classification report
print("Classification report ", metrics.classification_report(expected, predicted))

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = \n", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)

Classification report               precision    recall  f1-score   support

          1       0.73      0.76      0.75        54
          2       0.45      0.70      0.55        37
          3       0.35      0.39      0.37        49
          4       0.26      0.20      0.23        50
          5       0.43      0.32      0.37        60

avg / total       0.45      0.46      0.45       250

Confusion Metric = 
 [[41 11  1  1  0]
 [ 5 26  3  3  0]
 [ 4  7 19 12  7]
 [ 1 10 11 10 18]
 [ 5  4 20 12 19]]
Accuracy =  0.46
Sensitivity =  0.891304347826
Specificity =  0.702702702703
